In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'env/warming-443809753c14.json'

In [4]:
from google.cloud import bigquery
client = bigquery.Client()

In [5]:
data = pd.read_csv("https://lomudi-playground.s3.us-east-2.amazonaws.com/ds/GlobalLandTemperaturesByCity.csv")
data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [6]:
data_by_day = data.groupby(['dt', 'City', 'Latitude', 'Longitude'], sort=True)

In [7]:
data_by_day.size()

dt          City             Latitude  Longitude
1743-11-01  A Coruña         42.59N    8.73W        1
            Aachen           50.63N    6.34E        1
            Aalborg          57.05N    10.33E       1
            Aberdeen         57.05N    1.48W        1
            Aix En Provence  44.20N    4.47E        1
                                                   ..
2013-09-01  Århus            57.05N    10.33E       1
            Çorlu            40.99N    27.69E       1
            Çorum            40.99N    34.08E       1
            Öskemen          50.63N    82.39E       1
            Ürümqi           44.20N    87.20E       1
Length: 8599212, dtype: int64

In [8]:
data_by_day.first() 

AverageTemperature  \
dt         City            Latitude Longitude                       
1743-11-01 A Coruña        42.59N   8.73W                  10.779   
           Aachen          50.63N   6.34E                   6.425   
           Aalborg         57.05N   10.33E                  6.068   
           Aberdeen        57.05N   1.48W                   8.758   
           Aix En Provence 44.20N   4.47E                   7.478   
...                                                           ...   
2013-09-01 Århus           57.05N   10.33E                    NaN   
           Çorlu           40.99N   27.69E                    NaN   
           Çorum           40.99N   34.08E                    NaN   
           Öskemen         50.63N   82.39E                    NaN   
           Ürümqi          44.20N   87.20E                    NaN   

                                               AverageTemperatureUncertainty  \
dt         City            Latitude Longitude                                  
1743-11-01 A Coruña        42.59N   8.73W                              1.942   
           Aachen          50.63N   6.34E                              1.628   
           Aalborg         57.05N   10.33E                             1.737   
           Aberdeen        57.05N   1.48W                              1.886   
           Aix En Provence 44.20N   4.47E                              1.866   
...                                                                      ...   
2013-09-01 Århus           57.05N   10.33E                               NaN   
           Çorlu           40.99N   27.69E                               NaN   
           Çorum           40.99N   34.08E                               NaN   
           Öskemen         50.63N   82.39E                               NaN   
           Ürümqi          44.20N   87.20E                               NaN   

                                                      Country  
dt         City            Latitude Longitude                  
1743-11-01 A Coruña        42.59N   8.73W               Spain  
           Aachen          50.63N   6.34E             Germany  
           Aalborg         57.05N   10.33E            Denmark  
           Aberdeen        57.05N   1.48W      United Kingdom  
           Aix En Provence 44.20N   4.47E              France  
...                                                       ...  
2013-09-01 Århus           57.05N   10.33E            Denmark  
           Çorlu           40.99N   27.69E             Turkey  
           Çorum           40.99N   34.08E             Turkey  
           Öskemen         50.63N   82.39E         Kazakhstan  
           Ürümqi          44.20N   87.20E              China  

[8599212 rows x 3 columns]

In [9]:
#data_by_day.sum().reset_index().to_csv('global_temp_per_day_per_city.csv', index=False, header=True)

In [10]:
df = data_by_day.sum().reset_index()
df.head()

,dt,City,Latitude,Longitude,AverageTemperature,AverageTemperatureUncertainty
0,1743-11-01,A Coruña,42.59N,8.73W,10.779,1.942
1,1743-11-01,Aachen,50.63N,6.34E,6.425,1.628
2,1743-11-01,Aalborg,57.05N,10.33E,6.068,1.737
3,1743-11-01,Aberdeen,57.05N,1.48W,8.758,1.886
4,1743-11-01,Aix En Provence,44.20N,4.47E,7.478,1.866


In [11]:
df.to_csv('global_temp_per_day_per_city_sortred.tsv', sep = '\t', index=False, header=True)

In [12]:
df.to_csv('global_temp_per_day_per_city_sortred.csv', index=False, header=True)

In [13]:
table_id = 'warm_dataset.temp_by_city'

job = client.load_table_from_dataframe(
    df, table_id
)

job.result()